# Part 2: Automatic Ontology Generation

In [1]:
import spacy
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from collections import defaultdict

In [2]:
def filter_outcome_measures(file_path, encoding="utf-8"):
    filtered_measures = []
    with open(file_path, "r", encoding=encoding) as file:
        for line in file:
            line = line.strip()
            measures = line.split('|')
            for measure in measures:
                if len(measure.split()) <= 4 and 'change' not in measure.lower() and 'efficacy' not in measure.lower() and 'effectiveness' not in measure.lower() and 'time' not in measure.lower() and 'doctor' not in measure.lower() and 'physician' not in measure.lower() and 'health' not in measure.lower() and 'hospital' not in measure.lower() and 'baseline' not in measure.lower() and 'app' not in measure.lower() and 'device' not in measure.lower() and 'effect' not in measure.lower() and 'patients' not in measure.lower() and 'events' not in measure.lower():
                    filtered_measure = measure.replace('"', '')
                    filtered_measures.append(filtered_measure)

    # Remove duplicates from filtered_measures
    filtered_measures = list(set(filtered_measures))

    return filtered_measures


In [7]:
file_path = "small.txt"
filtered_measures = filter_outcome_measures(file_path)

print(filtered_measures)

['', 'Renal glucose clearance', 'Anthropometrics', 'body mass index', 'Reduction/elimination of hypoglycemic episodes', 'Glycemic Variability at Endpoint', 'Fall risk score', 'Cardiometabolic risk Z score', 'Hemoglobin A1c', 'Genetic Differences', 'Questionnaires', 'Insulin secretion', 'Diabetes Control', 'State Trait Anxiety Inventory', 'Presence of microalbuminuria', 'Glycated Haemoglobin concentration (mmol/mol)', 'Waist Circumference', 'Waist circumference', 'Total cholesterol', 'HDL-cholesterol', 'Dose regimen', 'Depression', 'Glycaemic Variability (Glucose levels)', 'lipid levels', 'Sit up test', 'Diabetes knowledge', 'BMI', 'Fasting plasma glucose levels', 'Urinary glucose concentration', 'Body Weight', '30sec sit-to-stand (30secSTS)', 'Glycated hemoglobin (HbA1c) levels', 'Glycemic variability', 'Parent Quality of Life', 'triglycerides', 'Overall stability index', 'Canagliflozin Treatment Adherence', 'Dietary carbohydrate intake', 'Bed days of care', 'Blood Pressure', 'HgAlc</=

### Concept categorization using Support Vector Machines (SVM)

In [12]:
# Loading the pre-trained spaCy model
nlp = spacy.load("en_core_web_sm")

# Training data (labeled concepts with their categories)
training_data = [
    ("Blood glucose", "Biomarker"),("Leptin", "Biomarker"),("HbA1c", "Biomarker"),("Insulin", "Biomarker"),("Free Fatty Acids", "Biomarker"),
    ("High-Sensivity R-Reactive Protein", "Biomarker"),("Homa Beta-cell Function (HOMA-B)", "Biomarker"),("Interleukin-18", "Biomarker"),
    ("Lipoproteins", "Biomarker"),("Plasma", "Biomarker"),("Plasminogen Activator Inhibitor-1", "Biomarker"),("Epinephrine", "Biomarker"),
    ("highsensitivity C-reactive protein", "Biomarker"),("hsCRP", "Biomarker"),("Adiponectin", "Biomarker"),("ApoB", "Biomarker"),("Waist Circumference", "Endpoint score"),
    ("Apolipoprotein B", "Biomarker"),("Blood Pressure", "Biomarker"),("Blood lipids", "Biomarker"),("C-peptide", "Biomarker"),("Adverse events", "Complications"),
    ("Cholesterol", "Biomarker"),("BMI", "Endpoint score"),("AUC", "Endpoint score"),("Rate", "Endpoint score"),("index", "Endpoint score"),("indices", "Endpoint score"),("Level", "Endpoint score"),("concentration", "Endpoint score"),
    ("Alloreactive Antibody Levels", "Endpoint score"),("Ankle Brachial Index", "Endpoint score"),("DASS-C21scale score", "Endpoint score"),("glucose levels", "Endpoint score"),
    ("Expiration/Inspiration Ratio", "Endpoint score"),("Fluid cell count", "Endpoint score"),("density", "Endpoint score"),("ratio", "Endpoint score"),("Variability", "Endpoint score"),
    ("Hyperglycemic Events Frequency", "Endpoint score"), ("Urinary Output", "Endpoint score"),("Diabetic retinopathy", "Complications"),("VO2max", "Endpoint score"),
    ("Cardiovascular disease", "Complications"),("Fatigue", "Complications"),("Depression", "Complications"),("Meal Tolerance Test", "Measurement tool"),
    ("Diabetic ketoacidosis", "Complications"),("Glucometer", "Measurement tool"),("Electromyography", "Measurement tool"),("metabolomics", "Measurement tool"),("biomedicalscreen", "Measurement tool"),
    ("1H-MRS", "Measurement tool"),("Antibody titer", "Measurement tool"),("CGM", "Measurement tool"),("Diabetes Numeracy Test", "Measurement tool"),("test", "Measurement tool"),("analysis", "Measurement tool"),
    ("Hyperinsulinemic-euglycemic clamp", "Measurement tool"),("Pharmacodynamics", "Measurement tool"),("Stadiometer", "Measurement tool"),("C-peptide levels", "Endpoint score"),
    ("Seca 203 Circumference Measuring Tape", "Measurement tool"),("Self-monitoring of blood glucose", "Measurement tool"),("MoCA", "Questionnaire"),("PHQ-9", "Questionnaire"),
    ("Quality of Life", "Questionnaire"),("DES-DSF", "Questionnaire"),("Diabetes distress survey", "Questionnaire"),("Questionnaire", "Questionnaire"),("survey", "Questionnaire"),
    ("Mini Nutritional Assessment", "Questionnaire"),("DTSQ", "Questionnaire"),("Confidence in Diabetes Self-Management", "Questionnaire"),("GAD-7", "Questionnaire"),
    ("Diabetes knowledge questionnaire", "Questionnaire"),("EuroQol 5D5L", "Questionnaire"),("General Anxiety Disorder-7", "Questionnaire"),("Knowledge", "Questionnaire"),
    ("Fear Survey II", "Questionnaire"),("Post Clinical Encounter", "Questionnaire"),("Device satisfaction questionaire", "Questionnaire"),("WHO-5", "Questionnaire"),
    ("Food Frequency Questionnaire", "Questionnaire"), ("Score", "Endpoint score"),("Twenty-four Hour Recall Interview", "Questionnaire"),("Questionnaires", "Questionnaire"),

]

# Extracting the concepts and their categories from the training data
concepts, categories = zip(*training_data)

# Adding an "Other" category
concepts += ("Other",)
categories += ("Other",)

# Creating a pipeline for concept categorization
pipeline = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("svm", svm.SVC(kernel="linear"))
])

# Fitting the pipeline to the training data
pipeline.fit(concepts, categories)

# Categorizing the concepts using the trained SVM model
predicted_categories = pipeline.predict(filtered_measures)
    
# Grouping concepts by categories
concepts_by_category = defaultdict(list)
for concept, category in zip(filtered_measures, predicted_categories):
    concepts_by_category[category].append(concept)

# Printing the categorized concepts by category
for category, concepts in concepts_by_category.items():
    concept_str = ", ".join(concepts)
    print(f"{category}: {concept_str}")

Biomarker: , Renal glucose clearance, Anthropometrics, Hemoglobin A1c, Genetic Differences, Insulin secretion, Total cholesterol, HDL-cholesterol, Dose regimen, Body Weight, 30sec sit-to-stand (30secSTS), triglycerides, Canagliflozin Treatment Adherence, Dietary carbohydrate intake, Blood Pressure, HgAlc</= 6.5%, LDL-cholesterol, Biering-SÃ¸rensen tests, Physical activity, LDL Cholesterol, Immunophenotyping, Perceived social support, Forced vital capacity (FVC), Safety, Hunger-satiety scores (millimeters), tests such as cholesterol, Insulin use </=0.2units/kg/day, Alcohol Misuse, diet, Cardiovascular structure and function, Glucose, Insulin Secretion, Hyperglycaemia Avoidance Scale, Urine volume, Body weight, Hemoglobin A1C, Social Support, Fasting Lipid Profile, Hypertension, HbA1c, FVC/FEV1, C peptide response, Fat-free mass, Glycoalbumin
Endpoint score: body mass index, Glycemic Variability at Endpoint, Fall risk score, Cardiometabolic risk Z score, Glycated Haemoglobin concentratio

In [15]:
# Loading the pre-trained spaCy model
nlp = spacy.load("en_core_web_sm")

# Converting the concepts into numerical features
vectorizer = TfidfVectorizer()
concept_features = vectorizer.fit_transform(filtered_measures)

# Performing K-means clustering
num_clusters = 5
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(concept_features)
  
# Creating a dictionary to store concepts by cluster
clustered_concepts = defaultdict(list)

# Assigning concepts to their respective clusters
for concept, label in zip(filtered_measures, kmeans.labels_):
    clustered_concepts[label].append(concept)

# Printing the concepts by cluster
for cluster, concepts in clustered_concepts.items():
    print(f"Cluster {cluster}: {', '.join(concepts)}")

Cluster 3: , Renal glucose clearance, Anthropometrics, body mass index, Glycemic Variability at Endpoint, Fall risk score, Cardiometabolic risk Z score, Hemoglobin A1c, Genetic Differences, Questionnaires, Insulin secretion, Diabetes Control, State Trait Anxiety Inventory, Glycated Haemoglobin concentration (mmol/mol), Waist Circumference, Waist circumference, Dose regimen, Depression, Glycaemic Variability (Glucose levels), lipid levels, Sit up test, Diabetes knowledge, BMI, Fasting plasma glucose levels, Urinary glucose concentration, Body Weight, 30sec sit-to-stand (30secSTS), Glycated hemoglobin (HbA1c) levels, Glycemic variability, triglycerides, Overall stability index, Canagliflozin Treatment Adherence, Dietary carbohydrate intake, Blood Pressure, HgAlc</= 6.5%, Fasting Glucose Level, Self-reported medication adherence, hemoglobin A1C levels, Biering-SÃ¸rensen tests, GAD-7, Physical activity, Immunophenotyping, Urinary glucose excretion rate, Forced vital capacity (FVC), Hunger-